In [186]:
using Revise
#using ProgressBars
using Random
using ITensors
#using Pkg;Pkg.activate("../.")
using RandomMeas
using StatsBase
using TimerOutputs
using ProgressMeter

## Shallow Shadows 
Based on https://quantum-journal.org/papers/q-2023-06-01-1026 and https://doi.org/10.1103/PhysRevResearch.5.023027 and arXiv:2402.17911

### Introduction
For a given set of random unitaries, the measurement channel is defined as $\mathcal{M}(\rho)=\sum_s \bra{s}u\rho u^\dagger \ket{s}E[u^\dagger \ket{s}\bra{s}u]$. To design a randomized measurement protocol, we need to learn this channel, then invert it. Once we know the inverse channel, we will be able to form shadows on arbitrary states $\rho$, as $\hat{\rho}=\mathcal{M}^{-1}(u^\dagger \ket{s}\bra{s}u)$

We begin by picking a set of $N_u$ unitaries and calculate numerically the measurement channel $\sigma_0=\mathcal{M}(\rho_0)$ from MPS simulations. Note that alternatively, one could consider measuring an approximate representation of the channel experimentally (arXiv:2402.17911)

In [216]:
#Step 1: Calibration Data acquisition
N = 6
ξ = siteinds("Qubit", N;addtags="output")
χ = 4
Nu = 1000
states = ["Dn" for n in 1:N]
ψ0 = MPS(ξ,states);
ρ0 = outer(ψ0',ψ0);

In [ ]:
u = Vector{Vector{ITensor}}()
for r in 1:Nu
    u1 = [op("RandomUnitary",ξ[i]) for i in 1:N]
    #push!(u,u1)
    u2 = [op("RandomUnitary",ξ[i],ξ[i+1]) for i in 1:N-1]
    u3 = [op("RandomUnitary",ξ[i],ξ[i+1]) for i in 2:N-2]
    push!(u,[u1;u2;u3])
end
M = EvaluateMeasurementChannel(ψ0,u);

Progress:  43%|█████████████████▌                       |  ETA: 0:00:36

In [ ]:
nsweeps = 4
σ0 = FitChannelMPO(M,χ,nsweeps);

### Inverting the channel
Now we can then find the inverse map $\mathcal{M}^{-1}(\rho)=\sum_A d_A \rho_A$, such that $\mathcal{M}^{-1}(\mathcal{M})\approx\mathbf{1}$. This is based on automatic-differentation library Zygote and minimizing the cost function $||\rho_0-\mathcal{M}^{-1}(\sigma_0)||^2=||\rho_0-\mathcal{M}^{-1}(\mathcal{M}(\rho_0))||^2$, where the vector $d$ is parametrized as MPS of bond dimension $\chi$

In [ ]:
d = InversionChannel(ρ0,σ0,χ);

## Data acquisition
We are ready to perform an experiment on a unknown state and build shadows as MPO

In [ ]:
using PastaQ
NM = 10
circuit = randomcircuit(N, depth=2)
#noisemodel1 = (1 => ("depolarizing", (p = 0,)),2 => ("depolarizing", (p = 0.05,)))
ψ = runcircuit(ψ0,circuit);

## Extracting observables
Here we focus on XX correlations

In [ ]:
O = Vector{MPO}()
for i in 1:N-1
    ampo = AutoMPO()
    ampo .+= "X", i,"X",i+1
    push!(O,MPO(ampo,ξ))
end
O_exact = zeros(N-1);

In [ ]:
O_est = zeros(N-1);
@showprogress dt=1 for r in 1:Nu
            data = get_RandomMeas(ψ,u[r],NM) #data acquisition in simulated quantum device
            shadow = get_ShallowShadow(data,u[r],d,ξ)
            for m in 1:NM
               for i in 1:N-1
                    #$X = apply(O[i],shadow[m],apply_dag=false)
                    O_est[i] += real(inner(O[i],shadow[m]))/Nu/NM
                end 
            end
end

In [ ]:
for i in 1:N-1
    O_exact[i] = real(inner(ψ',O[i],ψ))
    println("XX for pair  ",i,i+1)
    println("Exact ", O_exact[i])
    println("Estimated ", O_est[i])
end
